In [ ]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import sshutils
import mysecrets
import sqlite_db_utils
import rsutils.s2_grid_utils

In [ ]:
LOCAL_DATACUBE_FOLDERPATH = '../data/datacubes'

In [ ]:
LOCAL_DATACUBE_CATALOG_DB_PATH = sshutils.download_file_from_cluster(
    sshcreds = mysecrets.SSH_UMD,
    remotepath = '/gpfs/data1/cmongp2/sasirajann/fetch_satdata/data/datacubes/catalog.db',
    download_folderpath = LOCAL_DATACUBE_FOLDERPATH,
    overwrite = True,
)

try:
    sshutils.download_file_from_cluster(
        sshcreds = mysecrets.SSH_UMD,
        remotepath = '/gpfs/data1/cmongp2/sasirajann/fetch_satdata/data/datacubes/catalog.db-shm',
        download_folderpath = LOCAL_DATACUBE_FOLDERPATH,
        overwrite = True,
    )
    sshutils.download_file_from_cluster(
        sshcreds = mysecrets.SSH_UMD,
        remotepath = '/gpfs/data1/cmongp2/sasirajann/fetch_satdata/data/datacubes/catalog.db-wal',
        download_folderpath = LOCAL_DATACUBE_FOLDERPATH,
        overwrite = True,
    )
    print('wal and shm files downloaded')
except Exception as e:
    raise e

In [ ]:
sqlite_db_utils.get_tables_in_db(
    database = LOCAL_DATACUBE_CATALOG_DB_PATH,
)

In [ ]:
s2l2a_datacube_catalog_gdf = sqlite_db_utils.fetch_rows_from_db(
    database = LOCAL_DATACUBE_CATALOG_DB_PATH,
    table = 'sentinel-2-l2a',
    timestamp_cols = ['last_update', 'startdate', 'enddate']
)

In [ ]:
s2l2a_datacube_catalog_gdf.tail()

In [ ]:
s2l2a_datacube_catalog_gdf[s2l2a_datacube_catalog_gdf['roi'] == 's2grid=17b4eec']

In [ ]:
scale = 5
aspect_ratio = 3
fig, ax = plt.subplots(figsize=(scale*aspect_ratio, scale))

filtered_s2l2a_datacube_catalog_gdf = s2l2a_datacube_catalog_gdf[
    (s2l2a_datacube_catalog_gdf['roi'].str.startswith('s2grid=')) &
    (s2l2a_datacube_catalog_gdf['last_update'] > pd.Timestamp(2024, 2, 25, tz='UTC'))
]

g = sns.histplot(
    ax = ax,
    data = filtered_s2l2a_datacube_catalog_gdf,
    x = 'last_update',
    bins = 100,
)

In [ ]:
d = 50

last_updates = filtered_s2l2a_datacube_catalog_gdf['last_update'].sort_values().to_numpy()
t_deltas = np.array([
    (last_updates[i+d] - last_updates[i]).total_seconds()
    for i in range(0, last_updates.shape[0]-d)
])

print('shape =', t_deltas.shape)
print('min =', t_deltas.min())
print('max =', t_deltas.max())

In [ ]:
scale = 5
aspect_ratio = 3
fig, ax = plt.subplots(figsize=(scale*aspect_ratio, scale))

g = sns.lineplot(
    ax = ax,
    data = {
        'index': list(range(t_deltas.shape[0])),
        't_deltas': t_deltas,
        'datacubes per second': d / t_deltas,
        'last_update': last_updates[:t_deltas.shape[0]]
    },
    y = 'datacubes per second',
    # x = 'index'
    x = 'last_update',

)
g.grid()
g.set_title(f'Datacube creation rate (d={d})')

In [ ]:
s2grids_datacubes_present = s2l2a_datacube_catalog_gdf[
    s2l2a_datacube_catalog_gdf['roi'].str.startswith('s2grid=')
]['roi'].str.removeprefix('s2grid=').unique()

s2grids_datacubes_present

In [ ]:
gpd.GeoDataFrame(
    data = {
        's2grid': s2grids_datacubes_present,
        'geometry': [
            rsutils.s2_grid_utils.get_grid_geometry_from_id(grid_id=grid_id)
            for grid_id in s2grids_datacubes_present
        ],
    },
    crs = 'epsg:4326'
).to_file('../data/ethiopia/s2grids_with_datacubes.geojson')

In [ ]:
s2l2a_datacube_catalog_gdf['local_folderpath'].to_list()[-1]

### Get failed rois

In [ ]:
with open('../data/ethiopia/inference_rois_except_wolayita.txt') as f:
    inference_rois = [l[:-1] for l in f.readlines()]

len(inference_rois)

In [ ]:
s2l2a_datacube_catalog_gdf[s2l2a_datacube_catalog_gdf['roi'].isin(inference_rois)].to_csv('../data/ethiopia/model_deploy_input_exceptwolayita_2019-2021.csv', index=False)

In [ ]:
year_failed_rois_dict = dict()

for year in [2019, 2020, 2021]:
    year_failed_rois_dict[year] = set(inference_rois) - set(s2l2a_datacube_catalog_gdf[
        (s2l2a_datacube_catalog_gdf['startdate'] > pd.Timestamp(year, 3, 1, tz='UTC')) &
        (s2l2a_datacube_catalog_gdf['startdate'] < pd.Timestamp(year, 3, 10, tz='UTC'))
    ]['roi'])

year_failed_rois_dict

In [ ]:
def s2grids_list_to_gdf(
    s2gridids:list[str],
):
    data = {
        's2gridid': [],
        'geometry': [],
    }
    for s2gridid in s2gridids:
        geometry = rsutils.s2_grid_utils.get_grid_geometry_from_id(s2gridid)
        data['s2gridid'].append(s2gridid)
        data['geometry'].append(geometry)

    return gpd.GeoDataFrame(data=data, crs='epsg:4326')

In [ ]:
with open('../data/ethiopia/inference_rois.txt') as f:
    s2gridids = [line[:-1].removeprefix('s2grid=') for line in f.readlines()]

s2grids_list_to_gdf(s2gridids = s2gridids).to_file('../data/ethiopia/inference_rois.geojson')

In [13]:
1768 / 104 * (2*60 + 16 + 456.23) / 60

167.7985

In [12]:
((2*60 + 41) * 60 + 22) / 60

161.36666666666667

In [1]:
9902.17 / 60 / 60

2.750602777777778

In [2]:
0.75 * 60

45.0

In [14]:
1200.0 / 60

20.0

In [15]:
20 / 3

6.666666666666667

In [ ]:
1 + 30 + 32 + 32 + 9.6 + 11 + 11 + 40 + 1.5 + 11 + 9.9 + 11 + 771/1000

200.771

In [8]:
180 * 100 * 100 / (5 * 5) / 60

1200.0

In [ ]:
data = {
    'year': [],
    's2gridid': [],
    'geometry': [],
}

for year, rois in year_failed_rois_dict.items():
    for roi in rois:
        s2gridid = roi.removeprefix('s2grid=')
        geometry = rsutils.s2_grid_utils.get_grid_geometry_from_id(s2gridid)
        data['year'].append(year)
        data['s2gridid'].append(s2gridid)
        data['geometry'].append(geometry)

failed_rois_gdf = gpd.GeoDataFrame(data=data, crs='epsg:4326')
failed_rois_gdf.to_file('../data/ethiopia/failed_s2grids.geojson')

### Checking failed datacubes

In [ ]:
inputs_df = pd.read_csv('../data/ethiopia/inference2019_wolayita2020-2021_s2l2a_datacube_input.csv')
inputs_df['startdate'] = inputs_df['startdate'].apply(sqlite_db_utils.str_to_ts)
inputs_df['enddate'] = inputs_df['enddate'].apply(sqlite_db_utils.str_to_ts)

In [ ]:
failed_indexes = list(set(zip(
    inputs_df['roi'],
    inputs_df['startdate'],
    inputs_df['enddate'],
    inputs_df['config_id'],
)) - set(zip(
    s2l2a_datacube_catalog_gdf['roi'],
    s2l2a_datacube_catalog_gdf['startdate'],
    s2l2a_datacube_catalog_gdf['enddate'],
    s2l2a_datacube_catalog_gdf['config_id'],
)))
failed_indexes

In [ ]:
failed_inputs_df = \
inputs_df.set_index([
    'roi',
    'startdate',
    'enddate',
    'config_id',
]).loc[failed_indexes].reset_index()

In [ ]:
def get_shapes_gdf(roi:str):
    geom_type, geom_id = roi.split('=')
    if geom_type not in ['s2grid', 'geom']:
        raise ValueError(f'Invalid roi={roi}. roi must start with s2grid= or geom=')
    
    if geom_type == 's2grid':
        geom = rsutils.s2_grid_utils.get_grid_geometry_from_id(grid_id = geom_id)
    else:
        raise NotImplementedError()

    shapes_gdf = gpd.GeoDataFrame(
        data = {'geometry': [geom]},
        crs = 'epsg:4326',
    )

    return shapes_gdf

In [ ]:
failed_inputs_df['geometry'] = failed_inputs_df['roi'].apply(lambda x: get_shapes_gdf(x)['geometry'][0])

In [ ]:
failed_inputs_gdf = gpd.GeoDataFrame(failed_inputs_df, crs='epsg:4326')

In [ ]:
failed_inputs_gdf.to_file('../data/ethiopia/failed_inputs.geojson')

### Creating training datacube ids list

In [ ]:
with open('../data/ethiopia/training_datacube_ids.txt', 'w') as f:
    for _id in s2l2a_datacube_catalog_gdf[s2l2a_datacube_catalog_gdf['id'].str.startswith('geom=')]['id']:
        f.write(f'{_id}\n')

### Creating crops.npy

In [ ]:
import numpy as np
import geopandas as gpd

In [ ]:
data = np.load('../data/ethiopia/s2l2a_training_data/data.npy')
ids = np.load('../data/ethiopia/s2l2a_training_data/ids.npy', allow_pickle=True)[()]

In [ ]:
data.shape

In [ ]:
training_data_gdf = gpd.read_file('../data/ethiopia/normalised/combined.geojson')

In [ ]:
ids[0].split('=')[1].split('_')[:-3]

In [ ]:
training_data_gdf['id']

In [ ]:
id_crop_dict = dict(zip(
    training_data_gdf['id'],
    training_data_gdf['c_class'],
))

In [ ]:
crops = []
for _id in ids:
    _id = '_'.join(_id.split('=')[1].split('_')[:-3])
    crops.append(id_crop_dict[_id])

crops = np.array(crops)

In [ ]:
np.save('../data/ethiopia/s2l2a_training_data/crops.npy', crops)